In [ ]:
import cv2
import numpy as np
import math
import matplotlib.pyplot as plt
import random

In [ ]:
def extract_filter():
    pi_digits = [int(digit) for digit in str(math.pi)[2:6]]
    filter_values = [math.floor(d * 10 * math.pi) for d in pi_digits]
    filter_values.sort(reverse=True)
    return np.array(filter_values, dtype=np.uint8).reshape(2, 2)
filter_2x2 = extract_filter()
print(filter_2x2)

In [ ]:
def restore_portrait(distorted_image_path, filter_2x2):
    img = cv2.imread(distorted_image_path, cv2.IMREAD_GRAYSCALE)
    h, w = img.shape
    restored_img = img.copy()
    for i in range(0, h - 1, 2):
        for j in range(0, w - 1, 2):
            restored_img[i:i+2, j:j+2] = np.bitwise_xor(restored_img[i:i+2, j:j+2], filter_2x2)
    return restored_img
restored_img = restore_portrait("artwork_picasso.png", filter_2x2)
cv2.imwrite("restored.png", restored_img)

In [ ]:
def template_matching(collage_path, template_path):
    collage = cv2.imread(collage_path, cv2.IMREAD_GRAYSCALE)
    template = cv2.imread(template_path, cv2.IMREAD_GRAYSCALE)
    template = cv2.resize(template, (100, 100))
    h_temp, w_temp = template.shape
    h_collage, w_collage = collage.shape
    best_match = (0, 0)
    min_diff = float('inf')
    for i in range(h_collage - h_temp + 1):
        for j in range(w_collage - w_temp + 1):
            region = collage[i:i+h_temp, j:j+w_temp]
            diff = np.sum(np.abs(region.astype(int) - template.astype(int)))
            if diff < min_diff:
                min_diff = diff
                best_match = (j, i)
    return best_match
x, y = template_matching("collage.png", "restored.png")
print(f"Template found at coordinates: x={x}, y={y}")

In [ ]:
def compute_password(x, y):
    return math.floor((x + y) * math.pi)
password = compute_password(x, y)
print(f"Computed Password: {password}")

In [ ]:
def load_maze(filename):
    img = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)
    return img > 128
def is_collision(p1, p2, maze):
    x1, y1 = p1
    x2, y2 = p2
    num_steps = max(abs(x2 - x1), abs(y2 - y1))
    if num_steps == 0:
        return False
    for i in range(num_steps + 1):
        x = int(x1 + i * (x2 - x1) / num_steps)
        y = int(y1 + i * (y2 - y1) / num_steps)
        if not maze[y, x]:
            return True
    return False
class RRTConnect:
    def __init__(self, start, goal, maze, step_size=10, max_iter=5000):
        self.start = start
        self.goal = goal
        self.maze = maze
        self.step_size = step_size
        self.max_iter = max_iter
        self.tree_a = [start]
        self.tree_b = [goal]
        self.parent_a = {start: None}
        self.parent_b = {goal: None}
    def get_nearest(self, tree, point):
        nearest = min(tree, key=lambda p: np.linalg.norm(np.array(p) - np.array(point)))
        return nearest
    def extend(self, tree, point, parent):
        nearest = self.get_nearest(tree, point)
        direction = np.array(point) - np.array(nearest)
        distance = np.linalg.norm(direction)
        if distance == 0:
            return None
        if distance > self.step_size:
            direction = (direction / distance) * self.step_size
        new_point = tuple((np.array(nearest) + direction).astype(int))
        if 0 <= new_point[0] < self.maze.shape[1] and 0 <= new_point[1] < self.maze.shape[0]:
            if not is_collision(nearest, new_point, self.maze):
                tree.append(new_point)
                parent[new_point] = nearest
                return new_point
        return None
    def connect(self, tree, target, parent):
        while True:
            new_point = self.extend(tree, target, parent)
            if new_point is None or new_point == target:
                return new_point
    def find_path(self):
        iteration_count = 0
        for _ in range(self.max_iter):
            iteration_count += 1
            rand_point = (random.randint(0, self.maze.shape[1] - 1), random.randint(0, self.maze.shape[0] - 1))
            new_a = self.extend(self.tree_a, rand_point, self.parent_a)
            if new_a:
                new_b = self.connect(self.tree_b, new_a, self.parent_b)
                if new_b:
                    print(f"Path found in {iteration_count} iterations")
                    return self.build_path(new_a, new_b)
            self.tree_a, self.tree_b = self.tree_b, self.tree_a
            self.parent_a, self.parent_b = self.parent_b, self.parent_a
        return None
    def build_path(self, a, b):
        path = []
        while a:
            path.append(a)
            a = self.parent_a[a]
        path.reverse()
        while b:
            path.append(b)
            b = self.parent_b[b]
        return path
maze = load_maze("maze.png")
start, goal = (150, 10), (450, 350)
rrt = RRTConnect(start, goal, maze)
path = rrt.find_path()
plt.imshow(~maze, cmap='gray', origin='upper')
plt.scatter(*zip(*[start, goal]), c=['green', 'red'], s=50, label=['Start', 'Goal'])
if path:
    px, py = zip(*path)
    plt.plot(px, py, 'b-', linewidth=2, label='Path')
plt.legend()
plt.show()